Data from https://www.sleepdata.org/datasets/sleepaccel

In [1]:
import pandas as pd

In [35]:
accel = pd.read_csv("./data/1066528_acceleration.txt", sep=' ', header=None)
hr = pd.read_csv("./data/1066528_heartrate.txt", header=None)
labels = pd.read_csv("./data/1066528_labeled_sleep.txt", sep=' ', header=None)
steps = pd.read_csv("./data/1066528_steps.txt", header=None)

In [36]:
accel.columns = ['time', 'x', 'y', 'z']

In [37]:
accel['time'] = round(accel.time).astype(int).apply(lambda x: x//30)

In [38]:
accel.set_index('time', inplace=True)

In [39]:
labels.columns = ['time', 'label']

In [40]:
labels['time'] = round(labels.time).astype(int).apply(lambda x: x//30)

In [41]:
labels.set_index('time', inplace=True)

In [42]:
sleep = labels.join(accel)

In [43]:
hr.columns = ['time', 'hr']

In [44]:
hr['time'] = round(hr.time).astype(int).apply(lambda x: x//30)

In [45]:
hr.set_index('time', inplace=True)

In [46]:
sleep = sleep.join(hr)

In [47]:
steps.columns = ["time", "steps"]

In [48]:
steps['time'] = round(steps.time).astype(int).apply(lambda x: x//30)

In [3]:
def create_dataframes(base_path, id_list):
    dataframes = {}
    for id in id_list:
        try:
            # Read files in chunks
            chunksize = 10 ** 6
            sleep_df = pd.read_csv(f"{base_path}{id}_labeled_sleep.txt", header=None, chunksize=chunksize)
            acceleration_df = pd.read_csv(f"{base_path}{id}_acceleration.txt", sep=' ', header=None, chunksize=chunksize)
            heart_rate_df = pd.read_csv(f"{base_path}{id}_heartrate.txt", header=None, chunksize=chunksize)
            steps_df = pd.read_csv(f"{base_path}{id}_steps.txt", sep=' ', header=None, chunksize=chunksize)

            # Process each chunk
            merged_chunks = []
            for sleep_chunk, acceleration_chunk, heart_rate_chunk, steps_chunk in zip(sleep_df, acceleration_df, heart_rate_df, steps_df):
                # Automatically detect column names
                sleep_chunk.columns = ['time', 'label'] if len(sleep_chunk.columns) == 2 else list(range(len(sleep_chunk.columns)))
                acceleration_chunk.columns = ['time', 'x', 'y', 'z'] if len(acceleration_chunk.columns) == 4 else list(range(len(acceleration_chunk.columns)))
                heart_rate_chunk.columns = ['time', 'hr'] if len(heart_rate_chunk.columns) == 2 else list(range(len(heart_rate_chunk.columns)))
                steps_chunk.columns = ['time', 'steps'] if len(steps_chunk.columns) == 2 else list(range(len(steps_chunk.columns)))

                # Ensure 'time' column exists
                if 'time' not in sleep_chunk.columns:
                    sleep_chunk['time'] = range(len(sleep_chunk))
                if 'time' not in acceleration_chunk.columns:
                    acceleration_chunk['time'] = range(len(acceleration_chunk))
                if 'time' not in heart_rate_chunk.columns:
                    heart_rate_chunk['time'] = range(len(heart_rate_chunk))
                if 'time' not in steps_chunk.columns:
                    steps_chunk['time'] = range(len(steps_chunk))

                sleep_chunk['time'] = (sleep_chunk['time'].round().astype(int) // 30).astype(int)
                acceleration_chunk['time'] = (acceleration_chunk['time'].round().astype(int) // 30).astype(int)
                heart_rate_chunk['time'] = (heart_rate_chunk['time'].round().astype(int) // 30).astype(int)
                steps_chunk['time'] = (steps_chunk['time'].round().astype(int) // 30).astype(int)

                sleep_chunk.set_index('time', inplace=True)
                acceleration_chunk.set_index('time', inplace=True)
                heart_rate_chunk.set_index('time', inplace=True)
                steps_chunk.set_index('time', inplace=True)

                # Use suffixes to handle overlapping columns
                merged_chunk = sleep_chunk.join(acceleration_chunk, how='outer', lsuffix='_sleep', rsuffix='_acceleration')
                merged_chunk = merged_chunk.join(heart_rate_chunk, how='outer', lsuffix='_merged', rsuffix='_heart_rate')
                merged_chunk = merged_chunk.join(steps_chunk, how='outer', lsuffix='_merged', rsuffix='_steps')

                merged_chunks.append(merged_chunk)

            # Concatenate chunks
            merged_df = pd.concat(merged_chunks, axis=0)

            dataframes[id] = merged_df
        except FileNotFoundError:
            print(f"Files for ID {id} not found.")
    
    return dataframes

In [4]:
base_path = './data/'
id_list = [1066528, 844359, 781756]
dataframes = create_dataframes(base_path, id_list)
print(dataframes[0])

MemoryError: Unable to allocate 10.1 GiB for an array with shape (4, 338681905) and data type float64